# Workshop 102: Credit Risk Modeling for Business Students

## Business Scenario
You are working at a bank as a **credit risk analyst**.

Your job:
- Decide whether a customer should be approved or rejected for a loan.

Bad decisions have real costs:
- Approving risky customers → loan defaults
- Rejecting good customers → lost revenue

Machine Learning helps automate and improve this decision.


In [2]:
# Pandas = working with tables (like Excel)
import pandas as pd

# NumPy = numerical helpers
import numpy as np

# Scikit-learn = machine learning + utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Metrics = ways to evaluate model performance
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("✅ Imports loaded successfully!")

✅ Imports loaded successfully!


## Upload both CSV files (no Kaggle API)

Click the **Choose Files** button and upload **both**:
- `customer_data.csv`
- `payment_data.csv`


In [3]:
from google.colab import files

uploaded = files.upload()

print("Uploaded files:", list(uploaded.keys()))

Saving payment_data.csv to payment_data (1).csv
Saving customer_data.csv to customer_data (1).csv
Uploaded files: ['payment_data (1).csv', 'customer_data (1).csv']


In [4]:
# Load each CSV into a pandas DataFrame (a table)
customer_df = pd.read_csv("customer_data.csv")
payment_df  = pd.read_csv("payment_data.csv")

# Quick sanity checks
print("customer_df shape (rows, cols):", customer_df.shape)
print("payment_df shape  (rows, cols):", payment_df.shape)

customer_df shape (rows, cols): (1125, 13)
payment_df shape  (rows, cols): (8250, 12)


## Quick inspection

We check:
- first few rows
- column types
- whether missing values exist

This is like doing a quick “data audit” before modeling.


In [5]:
display(customer_df.head())
display(payment_df.head())

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000
3,1,58995168,7,1335.5,1,151000.0,2,11,5,110,3,60084,1.000000
4,0,54987320,7,NaN,2,59000.0,2,11,5,108,4,450081,197.403141


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016


In [6]:
print("---- customer_df.info() ----")
customer_df.info()

print("\n---- payment_df.info() ----")
payment_df.info()

---- customer_df.info() ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1125 non-null   int64  
 1   id      1125 non-null   int64  
 2   fea_1   1125 non-null   int64  
 3   fea_2   976 non-null    float64
 4   fea_3   1125 non-null   int64  
 5   fea_4   1125 non-null   float64
 6   fea_5   1125 non-null   int64  
 7   fea_6   1125 non-null   int64  
 8   fea_7   1125 non-null   int64  
 9   fea_8   1125 non-null   int64  
 10  fea_9   1125 non-null   int64  
 11  fea_10  1125 non-null   int64  
 12  fea_11  1125 non-null   float64
dtypes: float64(3), int64(10)
memory usage: 114.4 KB

---- payment_df.info() ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8250 entries, 0 to 8249
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               8250 n

## Target variable: `label`

In `customer_data.csv`, the column `label` is the **target**:
- `label = 0` → safe
- `label = 1` → risky (default)

Let’s check how many of each we have.


In [7]:
customer_df["label"].value_counts()

,count
label,
0,900
1,225


## Merge tables (SQL-style JOIN)

Real businesses store data in multiple tables.

Both tables share:
- `id` = customer ID

We merge customer info + payment behavior into one modeling table.


In [8]:
df = customer_df.merge(payment_df, on="id", how="inner")

print("Merged df shape (rows, cols):", df.shape)
df.head()

Merged df shape (rows, cols): (8250, 24)


,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,...,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,9,10,60500.0,31/01/2015,6657.6,18934.0,19/09/2015
1,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,18,10,NaN,28/04/2009,153792.0,149387.0,10/08/2015
2,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,0,0,0,1,5,NaN,30/03/2009,0.0,150500.0,NaN
3,1,54982665,5,1245.5,3,77000.0,2,15,5,109,...,2,26,11906,6,10,NaN,17/05/2007,0.0,46371.0,12/07/2013
4,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,...,0,0,0,4,6,NaN,02/06/2016,15120.0,21500.0,19/04/2016


## Drop date columns (keep it beginner-friendly)

Date columns like `update_date` and `report_date` are useful,
but require extra feature engineering.

For Workshop 102, we drop them to keep things simple.


In [9]:
date_cols = [c for c in ["update_date", "report_date"] if c in df.columns]
df = df.drop(columns=date_cols)

print("Dropped date columns:", date_cols)
print("New df shape:", df.shape)

Dropped date columns: ['update_date', 'report_date']
New df shape: (8250, 22)


## Handle missing values

ML models cannot handle missing values directly.

Strategy:
- Numeric columns → fill with median
- Categorical columns → fill with most common value

In [10]:
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df.select_dtypes(include=["object"]).columns

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print("Total missing values remaining:", df.isna().sum().sum())

Total missing values remaining: 0


## Features (X) and target (y)

We remove:
- `label` → the answer
- `id` → identifier only

The model should not see either.


In [11]:
X = df.drop(columns=["label", "id"])
y = df["label"]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (8250, 20)
y shape: (8250,)


## Encode categorical columns

Models like Logistic Regression need numeric inputs.

We use one-hot encoding to convert text into numbers.

In [12]:
X_encoded = pd.get_dummies(X, drop_first=False)

print("Before encoding:", X.shape)
print("After encoding :", X_encoded.shape)

Before encoding: (8250, 20)
After encoding : (8250, 20)


## Train/Test split

- 70% training data
- 30% testing data

`stratify=y` keeps safe/risky ratios consistent.


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes :", X_test.shape, y_test.shape)


Train shapes: (5775, 20) (5775,)
Test shapes : (2475, 20) (2475,)


## Scaling (Logistic Regression only)

Scaling ensures features with large values
don’t dominate smaller ones.

Important:
- Fit scaler on training data only
- Apply same scaler to test data


In [14]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("✅ Scaling complete!")

✅ Scaling complete!


## Model 1: Logistic Regression

A common baseline model in finance.


In [15]:
log_model = LogisticRegression(max_iter=2000)
log_model.fit(X_train_scaled, y_train)

print("✅ Logistic Regression trained!")

✅ Logistic Regression trained!


## Evaluate Logistic Regression

We care about:
- Accuracy
- Types of mistakes (confusion matrix)
- Precision & recall (business risk trade-offs)


In [16]:
y_pred_log = log_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_log))
print("\nClassification Report:\n", classification_report(y_test, y_pred_log))

Accuracy: 0.8327272727272728

Confusion Matrix:
 [[2056    3]
 [ 411    5]]

Classification Report:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      2059
           1       0.62      0.01      0.02       416

    accuracy                           0.83      2475
   macro avg       0.73      0.51      0.47      2475
weighted avg       0.80      0.83      0.76      2475



## Model 2: Decision Tree

Decision Trees behave like rules:
IF conditions → decision

They are easy to explain to business stakeholders.


In [17]:
tree_model = DecisionTreeClassifier(max_depth=4, random_state=42)
tree_model.fit(X_train, y_train)

print("✅ Decision Tree trained!")

✅ Decision Tree trained!


## Evaluate Decision Tree


In [18]:
y_pred_tree = tree_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_tree))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_tree))
print("\nClassification Report:\n", classification_report(y_test, y_pred_tree))


Accuracy: 0.8513131313131314

Confusion Matrix:
 [[2058    1]
 [ 367   49]]

Classification Report:
               precision    recall  f1-score   support

           0       0.85      1.00      0.92      2059
           1       0.98      0.12      0.21       416

    accuracy                           0.85      2475
   macro avg       0.91      0.56      0.56      2475
weighted avg       0.87      0.85      0.80      2475



## Feature importance (business insight)

Which features matter most in predicting risk?


In [19]:
importances = pd.Series(tree_model.feature_importances_, index=X_train.columns)
importances_sorted = importances.sort_values(ascending=False)

display(importances_sorted.head(15))

,0
fea_4,0.499324
fea_11,0.209968
fea_8,0.118266
fea_10,0.108349
fea_6,0.057355
prod_code,0.006737
fea_1,0.000000
fea_2,0.000000
fea_3,0.000000
fea_7,0.000000


## Mini hands-on activity

Change `max_depth` and re-run:
- Smaller → simpler & explainable
- Larger → more complex & possibly overfitting

In [20]:
tree_model2 = DecisionTreeClassifier(max_depth=6, random_state=42)
tree_model2.fit(X_train, y_train)

y_pred_tree2 = tree_model2.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_tree2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_tree2))
print("\nClassification Report:\n", classification_report(y_test, y_pred_tree2))


Accuracy: 0.8581818181818182

Confusion Matrix:
 [[2056    3]
 [ 348   68]]

Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92      2059
           1       0.96      0.16      0.28       416

    accuracy                           0.86      2475
   macro avg       0.91      0.58      0.60      2475
weighted avg       0.87      0.86      0.81      2475



## Resume-ready takeaway

**Credit Risk Modeling Project**
- Merged customer-level and payment-behavior datasets (SQL-style join)
- Trained Logistic Regression and Decision Tree models to predict loan default risk
- Evaluated performance using confusion matrices and precision/recall
- Identified key risk drivers using feature importance
